## NFL Predictions Model

2024 Results: https://docs.google.com/presentation/d/1V8nD894wQmctfSbW4x1LnBecPViLVMpCaN5Pzcu_H24/edit#slide=id.g322fb2b0fd6_0_7

2024 regular season record: 186-86

**Important**: .ipynb **MUST** be running in Python version **3.11** or under!

In [1]:
!python3 --version

Python 3.11.4


In [2]:
!pip install nfl_data_py



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install pandas
!pip install matplotlib


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import nfl_data_py as nfl
import pandas as pd
import os
import time
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [5]:

#upload nfl_elo data from 2020-2024 - Titled "nfl-win-totals_{insert year here}"
# from google.colab import files
# files.upload()

#VERY IMPORTANT: CHANGE YEAR
# current_year = 2024

from datetime import date, datetime

today = pd.to_datetime(datetime.today())
year = today.year
month = today.month
# day = today.day

current_season = year

if month <= 8:
    current_season = year - 1
elif month == 9:
    first_gameday = pd.to_datetime(nfl.import_schedules([year])['gameday'].head(1))
    if day <= (first_gameday - pd.Timedelta(days=2) + pd.Timedelta(hours=7)):
        current_season = year - 1

current_year = current_season


pbp_data = nfl.import_pbp_data(range(2019, current_year + 1))
# pbp_2022 = nfl.import_pbp_data([2022])
win_totals = nfl.import_schedules(range(2019, current_year + 1))



# pbp_clean_2021 = pbp_2021[(pbp_2021['play_type'] != 'qb_kneel') & (pbp_2021['play_type'] != 'no_play') & (pbp_2021['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
# pbp_rp_2021 = pbp_2021[((pbp_2021['pass'] == 1) | (pbp_2021['rush']==1)) & (pbp_2021['play_type'] != 'no_play')]
# fourthq_2021 = pbp_clean_2021[(pbp_clean_2021['qtr'] == 4) & (abs(pbp_clean_2021['score_differential']) <= 8 )]
# clutch_2021 = fourthq_2021[(fourthq_2021['game_seconds_remaining'] <= 300)]
# drive_2021 = pbp_2021[pbp_2021['drive_play_id_started'] == pbp_2021['play_id']]
# fourth_quarter_drives_2021 = drive_2021[(drive_2021['qtr']==4)&(abs(pbp_clean_2021['score_differential']) <= 8 )]


2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
2025 done.
Downcasting floats.


In [6]:

win_totals['away_win'] = win_totals.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
win_totals['home_win'] = win_totals.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
# home_wins = win_totals_2021.groupby('home_team')['home_win'].sum().reset_index()
# away_wins = win_totals_2021.groupby('away_team')['away_win'].sum().reset_index()
# wins_2021 = (home_wins['home_win'] + away_wins['away_win']).reset_index()
# wins_2021.drop(columns='index', inplace=True)
# wins_2021.rename(columns={0:'wins'},inplace=True)
# teams = home_wins[['home_team']].rename(columns={'home_team':'team'})
# wins_2021 = teams.join(wins_2021)
# wins_2021


In [7]:
###test
# win_totals_2021[win_totals_2021['season']==2022]
seasonal_data = nfl.import_seasonal_data(range(2019, current_year))
depth_chart = nfl.import_depth_charts(range(2019, current_year))
roster = nfl.import_seasonal_rosters(range(2019, current_year + 1))


In [8]:
seasonal_data[seasonal_data['season']==2019]

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,00-0019596,2019,REG,373,613,4057.0,24,8.0,27.0,185.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.178921
4,00-0020531,2019,REG,281,378,2979.0,27,4.0,12.0,89.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190652
6,00-0021206,2019,REG,3,5,24.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004985
7,00-0022127,2019,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.086170,0.286362,0.107915,0.133333,0.117904,0.119691,0.120624,0.112999,0.886097,0.083894
9,00-0022787,2019,REG,50,67,580.0,3,1.0,2.0,19.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,00-0035703,2019,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,0.040574
2437,00-0035704,2019,REG,100,156,1020.0,7,3.0,5.0,26.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.184162
2442,00-0035710,2019,REG,284,459,3027.0,24,12.0,38.0,295.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.185911
2450,00-0035719,2019,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.222588,0.428641,0.206808,0.107143,0.171123,0.162791,0.156975,0.186875,1.754923,0.129103


In [9]:
###feature engineer the training data


fullteamdata_arr=[]
for i in range(2019,current_year):
  pbp_temp = pbp_data[pbp_data['season']==i]
  # pbp_2022 = nfl.import_pbp_data([2022])
  win_totals_temp = win_totals[win_totals['season']==i]
  if i == 2019:
    win_totals_temp.replace(to_replace='OAK',value='LV',inplace=True)
  pbp_clean = pbp_temp[(pbp_temp['play_type'] != 'qb_kneel') & (pbp_temp['play_type'] != 'no_play') & (pbp_temp['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
  pbp_rp = pbp_temp[((pbp_temp['pass'] == 1) | (pbp_temp['rush']==1)) & (pbp_temp['play_type'] != 'no_play')]
  fourthq = pbp_clean[(pbp_clean['qtr'] == 4) & (abs(pbp_clean['score_differential']) <= 8 )]
  clutch_data = fourthq[(fourthq['game_seconds_remaining'] <= 300)]
  drive_data = pbp_temp[pbp_temp['drive_play_id_started'] == pbp_temp['play_id']]
  fourth_quarter_drives = drive_data[(drive_data['qtr']==4)&(abs(pbp_clean['score_differential']) <= 8 )]
  #calculate win totals
  home_wins = win_totals_temp.groupby('home_team')['home_win'].sum().reset_index()
  away_wins = win_totals_temp.groupby('away_team')['away_win'].sum().reset_index()
  wins_data = (home_wins['home_win'] + away_wins['away_win']).reset_index()
  wins_data.drop(columns='index', inplace=True)
  wins_data.rename(columns={0:'wins'},inplace=True)
  teams = home_wins[['home_team']].rename(columns={'home_team':'team'})
  wins_data = teams.join(wins_data)

  #off/def epa
  off_epa_data = pbp_clean.groupby('posteam')[['epa','wpa']].mean().reset_index()
  off_epa_data.rename(columns={'epa':'off_epa', 'wpa':'off_wpa', 'posteam':'team'}, inplace=True)
  def_epa_data = pbp_clean.groupby('defteam')[['epa', 'wpa']].mean().reset_index()
  def_epa_data.rename(columns={'epa':'def_epa', 'wpa':'def_wpa', 'defteam':'team'},inplace=True)
  epa_data = pd.merge(off_epa_data,def_epa_data,on='team')
  fullteamdata = epa_data
  #run/pass epa/wpa
  off_epa_rp_data = pbp_rp.groupby('posteam')[['epa', 'wpa']].mean().reset_index()
  off_epa_rp_data.rename(columns={'epa':'offrp_epa', 'wpa':'offrp_wpa', 'posteam':'team'}, inplace=True)
  def_epa_rp_data = pbp_rp.groupby('defteam')[['epa','wpa']].mean().reset_index()
  def_epa_rp_data.rename(columns={'epa':'defrp_epa', 'wpa':'defrp_wpa', 'defteam':'team'},inplace=True)
  epa_rp_data = pd.merge(off_epa_rp_data,def_epa_rp_data,on='team')
  fullteamdata = fullteamdata.merge(epa_rp_data, how='inner', on='team')

  #four_min_offense
  four_min_data = pbp_clean[((pbp_clean['half_seconds_remaining']<=240.0) & (pbp_clean['game_half'] == 'Half1')) | \
                                    (clutch_data['game_seconds_remaining']<=240.0)]
  off_epa_fourmin_data = four_min_data.groupby('posteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'off4min_epa', 'wpa':'off4min_wpa', 'posteam':'team'})
  def_epa_fourmin_data = four_min_data.groupby('defteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'def4min_epa', 'wpa':'def4min_wpa', 'defteam':'team'})
  fourmin_epa_data = pd.merge(off_epa_fourmin_data, def_epa_fourmin_data, on='team')
  fullteamdata = fullteamdata.merge(fourmin_epa_data, on='team')
  #4q
  fourthq = pbp_clean[(pbp_clean['qtr'] == 4) & (abs(pbp_clean['score_differential']) <= 8 )]
  #4thq_oneposs
  fourthq_off_epa_data = fourthq.groupby("posteam")[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'off4q_epa', 'wpa':'off4q_wpa', 'posteam':'team'})
  fourthq_def_epa_data = fourthq.groupby("defteam")[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'def4q_epa', 'wpa':'def4q_wpa', 'defteam':'team'})
  fourthq_epa_data = pd.merge(fourthq_off_epa_data, fourthq_def_epa_data, on='team')
  fullteamdata = fullteamdata.merge(fourthq_epa_data, on='team')
  #clutch
  clutch_data = fourthq[fourthq['game_seconds_remaining'] <= 300]
  clutch_off_epa_data = clutch_data.groupby('posteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'offclutch_epa', 'wpa':'offclutch_wpa', 'posteam':'team'})
  clutch_def_epa_data = clutch_data.groupby('defteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'defclutch_epa', 'wpa':'defclutch_wpa', 'defteam':'team'})
  clutch_epa_data = pd.merge(clutch_off_epa_data, clutch_def_epa_data, on='team')
  fullteamdata = fullteamdata.merge(clutch_epa_data, on='team')
  #downs
  first_down_data = pbp_rp[pbp_rp['down'] == 1]
  second_down_data = pbp_rp[pbp_rp['down'] == 2]
  third_down_data = pbp_rp[(pbp_rp['down'] == 3)]
  fourth_down_data = pbp_rp[pbp_rp['down']==4 ]


  #ydstogo
  second_down_off_ydstogo_data = second_down_data.groupby('posteam')['ydstogo'].mean().reset_index().rename(columns = {'ydstogo':'ydstogo_2nd_off', 'posteam':'team'})
  second_down_def_ydstogo_data = second_down_data.groupby('defteam')['ydstogo'].mean().reset_index().rename(columns = {'ydstogo':'ydstogo_2nd_def', 'defteam':'team'})

  third_down_off_ydstogo_data = third_down_data.groupby('posteam')['ydstogo'].mean().reset_index().rename(columns={'ydstogo':'ydstogo_3rd_off', 'posteam':'team'})
  third_down_def_ydstogo_data = third_down_data.groupby('defteam')['ydstogo'].mean().reset_index().rename(columns = {'ydstogo':'ydstogo_3rd_def', 'defteam':'team'})

  ydstogo2nd_data = pd.merge(second_down_off_ydstogo_data, second_down_def_ydstogo_data, on='team')
  ydstogo3rd_data = pd.merge(third_down_off_ydstogo_data, third_down_def_ydstogo_data, on='team')
  fullteamdata = fullteamdata.merge(ydstogo2nd_data, on='team')
  fullteamdata = fullteamdata.merge(ydstogo3rd_data, on='team')

  #3rd/4th down
  third_down_off_data = third_down_data.groupby('posteam')['third_down_converted'].mean().reset_index().rename(columns = {'third_down_converted':'third_down_off', 'posteam':'team'})
  fourth_down_off_data = fourth_down_data.groupby('posteam')['fourth_down_converted'].mean().reset_index().rename(columns={'fourth_down_converted':'fourth_down_off', 'posteam':'team'})
  third_down_def_data = third_down_data.groupby('defteam')['third_down_converted'].mean().reset_index().rename(columns={'third_down_converted':'third_down_def', 'defteam':'team'})
  fourth_down_def_data = fourth_down_data.groupby('defteam')['fourth_down_converted'].mean().reset_index().rename(columns={'fourth_down_converted':'fourth_down_def', 'defteam':'team'})
  third_down_conv_data = pd.merge(third_down_off_data, third_down_def_data, on='team')
  fourth_down_conv_data = pd.merge(fourth_down_off_data, fourth_down_def_data, on='team')
  fullteamdata = fullteamdata.merge(third_down_conv_data, on='team')
  fullteamdata = fullteamdata.merge(fourth_down_conv_data, on='team')
  fullteamdata
  #4q/clutch
  third_down_4q_data = third_down_data[(third_down_data['qtr'] == 4) & (abs(third_down_data['score_differential']) <= 8)]
  fourth_down_4q_data = fourth_down_data[(fourth_down_data['qtr'] == 4) & (abs(fourth_down_data['score_differential']) <= 8)]
  third_down_clutch_data = third_down_data[(third_down_data['game_seconds_remaining'] <= 300) & (abs(third_down_data['score_differential']) <= 8)]
  fourth_down_clutch_data = fourth_down_data[(fourth_down_data['game_seconds_remaining'] <= 300) & (abs(fourth_down_data['score_differential']) <= 8)]
  fullteamdata['third_down_off_4q'] = third_down_4q_data.groupby('posteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['third_down_def_4q'] = third_down_4q_data.groupby('defteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['third_down_off_clutch'] = third_down_clutch_data.groupby('posteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['third_down_def_clutch'] = third_down_clutch_data.groupby('defteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  #first down pass/run
  fullteamdata['first_down_passrun_off'] = (first_down_data.groupby('posteam').sum()['pass'] / first_down_data.groupby('posteam').sum()['rush']).reset_index()[0]
  fullteamdata['first_down_passrun_def'] = (first_down_data.groupby('defteam').sum()['pass'] / first_down_data.groupby('defteam').sum()['rush']).reset_index()[0]

  ### basic stats
  ##play_type
  #punt
  total_punt_data = pbp_clean[pbp_clean['play_type'] == 'punt'].groupby('posteam')['play_id'].count()
  fullteamdata['total_punt'] = total_punt_data.reset_index()['play_id']
  #goal_to_go
  fullteamdata['goal_to_go'] = pbp_clean.groupby('posteam')['goal_to_go'].sum().reset_index()['goal_to_go']
  fullteamdata['goal_to_go_def'] = pbp_clean.groupby('defteam')['goal_to_go'].sum().reset_index()['goal_to_go']
  #kickoffs
  fullteamdata['kickoff'] = pbp_clean[pbp_clean['play_type'] == 'kickoff'].groupby('defteam').count()['play_id'].reset_index()['play_id']
  fullteamdata['kickoff_received'] = pbp_clean[pbp_clean['play_type'] == 'kickoff'].groupby('posteam').count()['play_id'].reset_index()['play_id']
  #pass/run ratio
  fullteamdata['pass_run_ratio'] = (pbp_rp.groupby('posteam').sum()['pass'] / pbp_rp.groupby('posteam').sum()['rush']).reset_index()[0]
  fullteamdata['opp_pass_run_ratio'] = (pbp_rp.groupby('defteam').sum()['pass'] / pbp_rp.groupby('posteam').sum()['rush']).reset_index()[0]
  #number of plays
  playcount_data = pbp_clean.posteam.value_counts()
  fullteamdata['playcount'] = playcount_data.reset_index().posteam
  play_differential_data = playcount_data - pbp_clean.defteam.value_counts()
  fullteamdata['play_differential'] = play_differential_data.reset_index()[0]
  fullteamdata
  ##yardsperplay(off/def)
  fullteamdata['ydsperplay'] = pbp_rp.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperattempt'] = pbp_rp[pbp_rp['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperrush'] = pbp_rp[pbp_rp['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperplay'] = pbp_rp.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperattempt'] = pbp_rp[pbp_rp['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperrush'] = pbp_rp[pbp_rp['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #4q
  fullteamdata['ydsperplay4q'] = fourthq.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperattempt4q'] = fourthq[fourthq['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperrush4q'] = fourthq[fourthq['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperplay4q'] = fourthq.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperattempt4q'] = fourthq[fourthq['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperrush4q'] = fourthq[fourthq['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #clutch
  fullteamdata['ydsperplay_clutch'] = clutch_data.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperattempt_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperrush_clutch'] = clutch_data[clutch_data['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperplay_clutch'] = clutch_data.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperattempt_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperrush_clutch'] = clutch_data[clutch_data['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #YAC
  fullteamdata['off_YAC'] = pbp_rp.groupby('posteam')['yards_after_catch'].mean().reset_index()['yards_after_catch']
  fullteamdata['def_YAC'] = pbp_rp.groupby('defteam')['yards_after_catch'].mean().reset_index()['yards_after_catch']
  #average point differential(including 4th quarter/clutch)
  fullteamdata['off_pointdifferential'] = pbp_clean.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['def_pointdifferential'] = (pbp_clean.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  fullteamdata['off_pointdifferential_4q'] = fourthq.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['def_pointdifferential_4q'] = (fourthq.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  fullteamdata['off_pointdifferential_clutch'] = clutch_data.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['def_pointdifferential_clutch'] = (clutch_data.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  #completion percentage
  fullteamdata['off_comp_percentage'] = pbp_rp[pbp_rp['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['def_comp_percentage'] = pbp_rp[pbp_rp['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['off_comp_percentage_4q'] = fourthq[fourthq['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['def_comp_percentage_4q'] = fourthq[fourthq['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['off_comp_percentage_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['def_comp_percentage_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata
  #interceptions
  fullteamdata['off_interception_rate'] = pbp_rp[pbp_rp['pass'] == 1].groupby('posteam')['interception'].mean().reset_index()['interception']
  fullteamdata['def_interception_rate'] = pbp_rp[pbp_rp['pass'] == 1].groupby('defteam')['interception'].mean().reset_index()['interception']
  fullteamdata['off_interception_rate_4q'] = fourthq[fourthq['pass'] == 1].groupby('posteam')['interception'].mean().reset_index()['interception']
  fullteamdata['def_interception_rate_4q'] = fourthq[fourthq['pass'] == 1].groupby('defteam')['interception'].mean().reset_index()['interception']
  #fumble
  fullteamdata['off_fumble'] = pbp_clean.groupby('posteam')['fumble'].sum().reset_index()['fumble']
  fullteamdata['off_fumble_lost'] = pbp_clean.groupby('posteam')['fumble_lost'].sum().reset_index()['fumble_lost']
  fullteamdata['def_fumble_lost'] = pbp_clean.groupby('defteam')['fumble_lost'].sum().reset_index()['fumble_lost']
  fullteamdata['fumble_forced'] = pbp_clean.groupby('defteam')['fumble_forced'].sum().reset_index()['fumble_forced']
  #touchdown
  fullteamdata['touchdown_diff'] = (pbp_clean.groupby('posteam')['touchdown'].sum() - pbp_clean.groupby('defteam')['touchdown'].sum()).reset_index()['touchdown']
  fullteamdata['touchdown_diff_4q'] = (fourthq.groupby('posteam')['touchdown'].sum() - fourthq.groupby('defteam')['touchdown'].sum()).reset_index()['touchdown']
  fullteamdata['touchdownrate_off'] = (pbp_clean.groupby('posteam')['touchdown'].mean()).reset_index()['touchdown']
  fullteamdata['touchdownrate_def'] = (pbp_clean.groupby('defteam')['touchdown'].mean()).reset_index()['touchdown']
  #sack
  fullteamdata['sack_diff'] = (pbp_clean.groupby('defteam')['sack'].sum()-pbp_clean.groupby('posteam')['sack'].sum()).reset_index()['sack']
  fullteamdata['sack_diff4q'] = (fourthq.groupby('defteam')['sack'].sum()-fourthq.groupby('posteam')['sack'].sum()).reset_index()['sack']
  fullteamdata['sack_rate_pbp'] = pbp_rp.groupby('defteam')['sack'].mean().reset_index()['sack']
  fullteamdata['sackrate_def'] = pbp_rp[pbp_rp['pass']==1].groupby('defteam')['sack'].mean().reset_index()['sack']
  fullteamdata['sackrate_off'] = pbp_rp[pbp_rp['pass']==1].groupby('posteam')['sack'].mean().reset_index()['sack']
  #qbhit
  fullteamdata['qbhit_diff'] = (pbp_clean.groupby('defteam')['qb_hit'].sum()-pbp_clean.groupby('posteam')['qb_hit'].sum()).reset_index()['qb_hit']
  fullteamdata['qbhit_diff4q'] = (fourthq.groupby('defteam')['qb_hit'].sum()-fourthq.groupby('posteam')['qb_hit'].sum()).reset_index()['qb_hit']
  fullteamdata['qbhit_rate_pbp'] = pbp_rp.groupby('defteam')['qb_hit'].mean().reset_index()['qb_hit']
  fullteamdata['qbhitrate_def'] = pbp_rp[pbp_rp['pass']==1].groupby('defteam')['qb_hit'].mean().reset_index()['qb_hit']
  fullteamdata['qbhitrate_off'] = pbp_rp[pbp_rp['pass']==1].groupby('posteam')['qb_hit'].mean().reset_index()['qb_hit']

  ### advanced stats
  #win probability
  fullteamdata['off_winprob'] = pbp_clean.groupby('posteam')['wp'].mean().reset_index()['wp']
  fullteamdata['def_winprob'] = pbp_clean.groupby('defteam')['def_wp'].mean().reset_index()['def_wp']
  fullteamdata['off_winprob_fourthq'] = fourthq.groupby('posteam')['wp'].mean().reset_index()['wp']
  fullteamdata['def_winprob_fourthq'] = fourthq.groupby('defteam')['def_wp'].mean().reset_index()['def_wp']
  fullteamdata['off_winprob_clutch'] = clutch_data.groupby('posteam')['wp'].mean().reset_index()['wp']
  fullteamdata['def_winprob_clutch'] = clutch_data.groupby('defteam')['def_wp'].mean().reset_index()['def_wp']
  #series success
  fullteamdata['off_series_success'] = pbp_clean.groupby('posteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['def_series_success'] = pbp_clean.groupby('defteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['off_series_success_fourthq'] = fourthq.groupby('posteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['def_series_success_fourthq'] = fourthq.groupby('defteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['off_series_success_clutch'] = clutch_data.groupby('posteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['def_series_success_clutch'] = clutch_data.groupby('defteam')['series_success'].mean().reset_index()['series_success']
  clutch_data.groupby('defteam')['series_success'].mean().reset_index()
  #first down
  fullteamdata['firstdown_diff'] = (pbp_clean.groupby('posteam')['first_down'].sum()-pbp_clean.groupby('defteam')['first_down'].sum()).reset_index()['first_down']
  fullteamdata['firstdownrate_off'] = pbp_clean.groupby('posteam')['first_down'].mean().reset_index()['first_down']
  fullteamdata['firstdownrate_def'] = pbp_clean.groupby('defteam')['first_down'].mean().reset_index()['first_down']
  fullteamdata['firstdownrate_diff4q'] = (fourthq.groupby('posteam')['first_down'].mean()-fourthq.groupby('defteam')['first_down'].mean()).reset_index()['first_down']
  fullteamdata['firstdownrate_diffclutch'] = (clutch_data.groupby('posteam')['first_down'].mean()-clutch_data.groupby('defteam')['first_down'].mean()).reset_index()['first_down']
  #redzone/goaltogo
  fullteamdata['red_zone_off'] = (drive_data[(drive_data['drive_inside20']==1)&(drive_data['fixed_drive_result']=='Touchdown')].groupby('posteam')['play_id'].count() /drive_data.groupby('posteam')['drive_inside20'].sum()).reset_index()[0]
  fullteamdata['red_zone_def'] = (drive_data[(drive_data['drive_inside20']==1)&(drive_data['fixed_drive_result']=='Touchdown')].groupby('defteam')['play_id'].count()/drive_data.groupby('defteam')['drive_inside20'].sum()).reset_index()[0]
  fullteamdata['goaltogo_off'] = pbp_clean[pbp_clean['goal_to_go']==1].groupby('posteam').series_success.mean().reset_index()['series_success']
  fullteamdata['goaltogo_def'] = pbp_clean[pbp_clean['goal_to_go']==1].groupby('defteam').series_success.mean().reset_index()['series_success']

  ### special teams
  #punt_block_difference
  punt_block_data = (pbp_clean.groupby('posteam').punt_blocked.sum() - pbp_clean.groupby('defteam').punt_blocked.sum()).reset_index()\
  .rename(columns={'posteam':'team', 'punt_blocked':'punt_blocked_diff'})
  fullteamdata['punt_block'] = punt_block_data.punt_blocked_diff
  #punt_inside_20 proportion
  punt_inside20_data = (pbp_clean[pbp_clean['play_type'] == 'punt'].groupby('posteam')['punt_inside_twenty'].sum() / total_punt_data).reset_index()\
  .rename(columns={'posteam':'team', 0:'punt_inside_twenty'})
  fullteamdata['punt_inside20'] = punt_inside20_data.punt_inside_twenty
  fullteamdata
  #avg starting field position (exclude kickoffs)
  drive_nokickoff_data = drive_data[drive_data['play_type']!='kickoff']
  drive_avgstartoff_data = drive_nokickoff_data.groupby('posteam')[['yardline_100']].mean().reset_index()
  fullteamdata['drive_avgstart_off'] = drive_avgstartoff_data['yardline_100']
  drive_avgstartdef_data = drive_nokickoff_data.groupby('defteam')[['yardline_100']].mean().reset_index()
  fullteamdata['drive_avgstart_def'] = drive_avgstartdef_data['yardline_100']
  #avg field position
  avgfieldposoff_data = pbp_clean.groupby('posteam')[['yardline_100']].mean().reset_index()
  avgfieldposdef_data = pbp_clean.groupby('defteam')[['yardline_100']].mean().reset_index()
  fullteamdata['avg_fieldpos_off'] = avgfieldposoff_data['yardline_100']
  fullteamdata['avg_fieldpos_def'] = avgfieldposdef_data['yardline_100']
  #field goals
  field_goal_data = (drive_data[drive_data['fixed_drive_result'] == 'Field goal'].groupby('posteam').count()['play_id'] / pbp_clean.groupby('posteam')['field_goal_attempt'].sum()).reset_index()
  fullteamdata['field_goal'] = field_goal_data[0]
  ###penalties
  penalties_data = pbp_temp.groupby('penalty_team').count()['play_id'].reset_index()
  fullteamdata['penalties'] = penalties_data['play_id']
  penaltyyards_data = pbp_temp.groupby('penalty_team').sum()['penalty_yards'].reset_index()
  fullteamdata['penalty_yards'] = penaltyyards_data['penalty_yards']
  #third/fourth down
  penalties_3rdor4thdown_data = pbp_temp[(pbp_temp['down'] == 3) | (pbp_temp['down'] == 4)].groupby('penalty_team').count()['play_id'].reset_index()
  fullteamdata['penalties_bigdowns'] = penalties_3rdor4thdown_data['play_id']
  penaltyyards_3rdor4thdown_data = pbp_temp[(pbp_temp['down'] == 3) | (pbp_temp['down'] == 4)].groupby('penalty_team').sum()['penalty_yards'].reset_index()
  fullteamdata['penaltyyards_bigdowns'] = penaltyyards_3rdor4thdown_data['penalty_yards']

  ### per drive data
  #drivecount
  off_drive_count_data = drive_data.groupby('posteam')['play_id'].count().reset_index().rename(columns={'play_id':'drive_count', 'posteam':'team'})
  def_drive_count_data = drive_data.groupby('defteam')['play_id'].count().reset_index().rename(columns={'play_id':'drive_count', 'posteam':'team'})
  off_fourthquarter_drivecount_data = fourth_quarter_drives.groupby('posteam')['play_id'].count().reset_index()
  def_fourthquarter_drivecount_data = fourth_quarter_drives.groupby('defteam')['play_id'].count().reset_index()
  fullteamdata['drive_count'] = off_drive_count_data['drive_count']
  #plays per drive, first downs per drive, drive inside 20, drive ended with score
  fullteamdata[['off_plays_perdrive','off_firstdowns_perdrive','off_drive_inside20','off_drive_endedwithscore']] = drive_data.groupby('posteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  fullteamdata[['def_plays_perdrive','def_firstdowns_perdrive','def_drive_inside20','def_drive_endedwithscore']] = drive_data.groupby('defteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  fullteamdata[['off_plays_perdrive_4q','off_firstdowns_perdrive_4q','off_drive_inside20_4q','off_drive_endedwithscore_4q']] = drive_data[drive_data['qtr']==4].groupby('posteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  fullteamdata[['def_plays_perdrive_4q','def_firstdowns_perdrive_4q','def_drive_inside20_4q','def_drive_endedwithscore_4q']] = drive_data[drive_data['qtr']==4].groupby('defteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  #yards per drive
  fullteamdata['off_yards_per_drive'] = (pbp_clean.groupby('posteam')['yards_gained'].sum().reset_index()['yards_gained']/off_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['off_yards_per_drive_4q'] = (fourthq.groupby('posteam')['yards_gained'].sum().reset_index()['yards_gained']/off_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  fullteamdata['def_yards_per_drive'] = (pbp_clean.groupby('defteam')['yards_gained'].sum().reset_index()['yards_gained']/def_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['def_yards_per_drive4q'] = (fourthq.groupby('defteam')['yards_gained'].sum().reset_index()['yards_gained']/def_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  #touchdowns per drive
  fullteamdata['off_touchdowns_perdrive'] = (pbp_clean.groupby('posteam')['touchdown'].sum().reset_index()['touchdown']/off_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['def_touchdowns_perdrive'] = (pbp_clean.groupby('defteam')['touchdown'].sum().reset_index()['touchdown']/def_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['off_touchdowns_perdrive_4q'] = (fourthq.groupby('posteam')['touchdown'].sum().reset_index()['touchdown']/off_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  fullteamdata['def_touchdowns_perdrive_4q'] = (fourthq.groupby('defteam')['touchdown'].sum().reset_index()['touchdown']/def_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  #unsuccessful drives per drive
  fullteamdata['off_bad_drives_perdrive'] = (drive_data[(drive_data['fixed_drive_result']!='Touchdown') & (drive_data['fixed_drive_result']!='Field Goal')].groupby('posteam')['play_id'].count().reset_index()['play_id']/off_drive_count_data['drive_count'])
  fullteamdata['def_bad_drives_perdrive'] = (drive_data[(drive_data['fixed_drive_result']!='Touchdown') & (drive_data['fixed_drive_result']!='Field Goal')].groupby('defteam')['play_id'].count().reset_index()['play_id']/def_drive_count_data['drive_count'])
  fullteamdata['off_bad_drives_perdrive_4q'] = (fourth_quarter_drives[(fourth_quarter_drives['fixed_drive_result']!='Touchdown') & (fourth_quarter_drives['fixed_drive_result']!='Field Goal')].groupby('posteam')['play_id'].count().reset_index()['play_id']/off_fourthquarter_drivecount_data['play_id'])
  fullteamdata['def_bad_drives_perdrive_4q'] = (fourth_quarter_drives[(fourth_quarter_drives['fixed_drive_result']!='Touchdown') & (fourth_quarter_drives['fixed_drive_result']!='Field Goal')].groupby('defteam')['play_id'].count().reset_index()['play_id']/def_fourthquarter_drivecount_data['play_id'])

  ###seasonal dataframes (per game and per drive stats)
  drive_count_data = drive_data.groupby('posteam')['play_id'].count().reset_index().rename(columns={'play_id':'drive_count', 'posteam':'team'})
  game_count_data = win_totals_temp.groupby('home_team')['week'].nunique() + win_totals_temp.groupby('away_team')['week'].nunique()
  one_score_data = win_totals_temp[abs(win_totals_temp['result']) <=8]
  reg_data = win_totals_temp[win_totals_temp['game_type']=='REG']
  groupby_home = win_totals_temp.groupby('home_team')
  groupby_away = win_totals_temp.groupby('away_team')
  groupby_home_onescore = one_score_data.groupby('home_team')
  groupby_away_onescore = one_score_data.groupby('away_team')
  groupby_home_reg = reg_data.groupby('home_team')
  groupby_away_reg = reg_data.groupby('away_team')
  reggame_count_data = groupby_home_reg['week'].nunique()+groupby_away_reg['week'].nunique()

  #point differential
  fullteamdata['pointdifferential'] = (win_totals_temp.groupby('home_team')['result'].sum() + (win_totals_temp.groupby('away_team')['result'].sum()*-1)).reset_index()['result']
  fullteamdata['pointdifferential_onescore'] = (one_score_data.groupby('home_team')['result'].sum() + (one_score_data.groupby('away_team')['result'].sum()*-1)).reset_index()['result']
  #win percentage
  fullteamdata['win_percentage']=wins_data['wins']/((groupby_home['week'].nunique() + groupby_away['week'].nunique()).reset_index()['week'])
  fullteamdata['win_percentage_reg']=((groupby_home_reg['home_win'].sum()+groupby_away_reg['away_win'].sum())/\
  ((groupby_home_reg['week'].nunique() + groupby_away_reg['week'].nunique()))).reset_index()[0]
  fullteamdata['win_percentage_onescore']=((groupby_home_onescore['home_win'].sum()+groupby_away_onescore['away_win'].sum())/(groupby_home_onescore['week'].nunique()+groupby_away_onescore['week'].nunique())).reset_index()[0]
  fullteamdata
  #spread line
  fullteamdata['avg_spreadline'] = (groupby_home['spread_line'].mean()-groupby_away['spread_line'].mean()).reset_index()['spread_line']
  #points per game
  fullteamdata['points_pergame_off'] = ((groupby_home['home_score'].sum()+groupby_away['away_score'].sum())/game_count_data).reset_index()[0]
  fullteamdata['points_pergame_def'] = ((groupby_home['away_score'].sum()+groupby_away['home_score'].sum())/game_count_data).reset_index()[0]
  fullteamdata['points_pergame_off_reg'] = ((groupby_home_reg['home_score'].sum()+groupby_away_reg['away_score'].sum())/reggame_count_data).reset_index()[0]
  fullteamdata['points_pergame_def_reg'] = ((groupby_home_reg['away_score'].sum()+groupby_away_reg['home_score'].sum())/reggame_count_data).reset_index()[0]
  #yards per game
  fullteamdata['yds_pergame_off'] = (pbp_clean.groupby('posteam')['yards_gained'].sum()/game_count_data).reset_index()[0]
  fullteamdata['yds_pergame_def'] = (pbp_clean.groupby('defteam')['yards_gained'].sum()/game_count_data).reset_index()[0]
  fullteamdata['yds_pergame_off_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('posteam')['yards_gained'].sum()/reggame_count_data).reset_index()[0]
  fullteamdata['yds_pergame_def_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('defteam')['yards_gained'].sum()/reggame_count_data).reset_index()[0]
  fullteamdata
  #first downs per game
  fullteamdata['firstdowns_pergame_off'] = (pbp_clean.groupby('posteam')['first_down'].sum()/game_count_data).reset_index()[0]
  fullteamdata['firstdowns_pergame_def'] = (pbp_clean.groupby('defteam')['first_down'].sum()/game_count_data).reset_index()[0]
  fullteamdata['firstdowns_pergame_off_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('posteam')['first_down'].sum()/reggame_count_data).reset_index()[0]
  fullteamdata['firstdowns_pergame_def_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('defteam')['first_down'].sum()/reggame_count_data).reset_index()[0]

  ### post-thanksgiving stats
  post_thanksgiving = pbp_clean[pbp_clean['week']>=13]
  post_thanksgiving_4q = fourthq[pbp_clean['week']>=13]
  groupby_thanksgiving_home = win_totals_temp[win_totals_temp['week']>=13].groupby('home_team')
  groupby_thanksgiving_away = win_totals_temp[win_totals_temp['week']>=13].groupby('away_team')
  game_count_thanksgiving = groupby_thanksgiving_home['week'].nunique()+groupby_thanksgiving_away['week'].nunique()
  #yards per play(including fourth quarter)
  fullteamdata['postthxgiving_ydsperplay'] = post_thanksgiving.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_ydsperattempt'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_ydsperrush'] = post_thanksgiving[post_thanksgiving['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperplay'] = post_thanksgiving.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperattempt'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperrush'] = post_thanksgiving[post_thanksgiving['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #4q
  fullteamdata['postthxgiving_ydsperplay4q'] = post_thanksgiving_4q.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperplay4q'] = post_thanksgiving_4q.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #epa and wpa(including 4q)
  fullteamdata[['postthxgiving_epa_off','postthxgiving_wpa_off']] = post_thanksgiving.groupby('posteam')[['epa','wpa']].mean().reset_index()[['epa','wpa']]
  fullteamdata[['postthxgiving_epa_def','postthxgiving_wpa_def']] = def_epa_data = post_thanksgiving.groupby('defteam')[['epa', 'wpa']].mean().reset_index()[['epa','wpa']]
  fullteamdata[['postthxgiving_epa_off_4q','postthxgiving_wpa_off_4q']] = post_thanksgiving_4q.groupby("posteam")[['epa', 'wpa']].mean().reset_index()[['epa','wpa']]
  fullteamdata[['postthxgiving_epa_def_4q','postthxgiving_wpa_def_4q']] = post_thanksgiving_4q.groupby("defteam")[['epa', 'wpa']].mean().reset_index()[['epa','wpa']]
  #down stats
  postthanksgiving_seconddown = post_thanksgiving[post_thanksgiving['down'] == 2]
  postthanksgiving_thirddown = post_thanksgiving[(post_thanksgiving['down'] == 3)]
  fullteamdata['postthxgiving_2ndydstogo_off'] = postthanksgiving_seconddown.groupby('posteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_2ndydstogo_def'] = postthanksgiving_seconddown.groupby('defteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_3rdydstogo_off'] = postthanksgiving_thirddown.groupby('posteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_3rdydstogo_def'] = postthanksgiving_thirddown.groupby('defteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_3rddownconverted_off'] = postthanksgiving_thirddown.groupby('posteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['postthxgiving_3rddownconverted_def'] = postthanksgiving_thirddown.groupby('defteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  #points per game
  fullteamdata['postthxgiving_points_pergame_off'] = ((groupby_thanksgiving_home['home_score'].sum()+groupby_thanksgiving_away['away_score'].sum())/game_count_thanksgiving).reset_index()[0]
  fullteamdata['postthxgiving_points_pergame_def'] = ((groupby_thanksgiving_home['away_score'].sum()+groupby_thanksgiving_away['home_score'].sum())/game_count_thanksgiving).reset_index()[0]
  #yards per game
  fullteamdata['postthxgiving_yards_pergame_off'] = ((post_thanksgiving.groupby('posteam')['yards_gained'].sum())/game_count_thanksgiving).reset_index()[0]
  fullteamdata['postthxgiving_yards_pergame_def'] = ((post_thanksgiving.groupby('defteam')['yards_gained'].sum())/game_count_thanksgiving).reset_index()[0]
  #win percentage
  fullteamdata['postthxgiving_win_percentage']=((groupby_thanksgiving_home['home_win'].sum()+groupby_thanksgiving_away['away_win'].sum())/game_count_thanksgiving).reset_index()[0]
  #average point differential
  fullteamdata['postthxgiving_off_pointdifferential'] = post_thanksgiving.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['postthxgiving_def_pointdifferential'] = (post_thanksgiving.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  #completion percentage
  fullteamdata['postthxgiving_off_comp_percentage'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['postthxgiving_def_comp_percentage'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']

  ### account for qb offseason changes
  win_totals_2022 = win_totals[win_totals['season']==i+1]
  seasonal_temp = seasonal_data[seasonal_data['season']==i]
  if i == current_year-1:
    depth_chart_temp = depth_chart[depth_chart['season']==current_year-1]
  else:
    depth_chart_temp = depth_chart[depth_chart['season']==i+1]

  roster_temp = roster[(roster['season']==i)|(roster['season']==i+1)]
  qb1 = depth_chart_temp[(depth_chart_temp['depth_position']=='QB')&(depth_chart_temp['depth_team']=='1')&(depth_chart_temp['week']==1)][['club_code','gsis_id','full_name']]
  qb1 = qb1.drop_duplicates()
  #check for rookies
  qb1_transpose = qb1
  qb1_transpose=qb1_transpose.set_index('club_code')
  qb1_transpose = qb1_transpose.transpose()
  count=0
  for a in qb1_transpose:
    count+=1
    if qb1_transpose.loc['gsis_id',a] not in list(roster[roster['season']==i]['player_id']):
      #set brian hoyer as default
      qb1_transpose[a] = ['00-0026625','Brian Hoyer']
  qb1_transpose=qb1_transpose.transpose()
  qb1_seasonstats = seasonal_temp.set_index('player_id').filter(qb1_transpose['gsis_id'], axis=0)
  qb1.set_index('gsis_id',inplace=True)
  qb1_seasonstats['2022_team'] = qb1['club_code']
  qb1_seasonstats = qb1_seasonstats.reset_index().set_index('2022_team')
  qb1_seasonstats.sort_index(inplace=True)
  qb1_seasonstats = qb1_seasonstats.transpose()
  #fill in default stats
  for b in range(0,32):
    if qb1_seasonstats.iloc[4, b] < 75:
      qb1_seasonstats.iloc[4,b]=90
      qb1_seasonstats.iloc[3,b]=53
      qb1_seasonstats.iloc[5,b]=560
      qb1_seasonstats.iloc[6,b]=5
      qb1_seasonstats.iloc[7,b]=2
      qb1_seasonstats.iloc[15,b]=1.0
  qb1_seasonstats=qb1_seasonstats.transpose()
  ##account for rookies!!!
  #epa/db difference

  fullteamdata['epa/db_diff'] = ((qb1_seasonstats['passing_epa']/qb1_seasonstats['attempts']) - pbp_rp[pbp_rp['pass']==1].groupby('posteam')['epa'].mean()).reset_index()[0]
  qb1_seasonstats.reset_index(inplace=True)
  #completion percentage
  fullteamdata['comppct_diff'] = ((qb1_seasonstats['completions']/qb1_seasonstats['attempts']) - fullteamdata['off_comp_percentage']).reset_index()[0]
  #pass touchdowns minus interceptions
  fullteamdata['td-int_diff'] = ((qb1_seasonstats['passing_tds']-qb1_seasonstats['interceptions']) - (pbp_rp[pbp_rp['pass']==1].groupby('posteam')['touchdown'].sum()-\
                                      pbp_rp[pbp_rp['pass']==1].groupby('posteam')['interception'].sum()).reset_index()[0]).reset_index()[0]
  # yards per attempt
  fullteamdata['ypa_diff'] = ((qb1_seasonstats['passing_yards']/qb1_seasonstats['attempts']) - (pbp_rp[pbp_rp['pass']==1].groupby('posteam')['yards_gained'].mean()).reset_index()['yards_gained']).reset_index()[0]
  #fill in default data for qbs who got few snaps last season

  # nfelo data
#   nfelo_wintotals = ''
#   nfelo_wintotals = pd.read_csv(f"nfl-win-totals_{i+1}.csv")
#   nfelo_wintotals = nfelo_wintotals[['team','line','line_adj','over_odds','under_odds','wt_rating','sos_projected']].sort_values(by='team').reset_index(drop=True)
#   nfelo_wintotals=nfelo_wintotals.transpose()
#   #swap lac and lar so that it is consistent with the var fullteamdata
#   lac = nfelo_wintotals[16]
#   nfelo_wintotals[16] = nfelo_wintotals[17]
#   nfelo_wintotals[17]=lac
#   nfelo_wintotals=nfelo_wintotals.transpose()
#   nfelo_wintotals
#   fullteamdata = fullteamdata.join(nfelo_wintotals.iloc[:,1:])

  #average experience
  fullteamdata['avg_exp_2021'] = (roster[roster['season']==i].groupby('team')['years_exp'].mean()).reset_index()['years_exp']
  fullteamdata['avg_exp_2022'] = roster[roster['season']==i+1].groupby('team')['years_exp'].mean().reset_index()['years_exp']
  fullteamdata['exp_diff'] = fullteamdata['avg_exp_2021']-fullteamdata['avg_exp_2022']

  #wins
  fullteamdata['wins'] = wins_data['wins']


  fullteamdata_arr.append(fullteamdata)
# fullteamdata
# for i in fullteamdata:
#   i.dropna()
#   print(i.shape)
fullteamdata
# win_totals_2021.groupby('home_team')['week'].nunique() + win_totals_2021.groupby('away_team')['week'].nunique()

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win_totals_temp.replace(to_replace='OAK',value='LV',inplace=True)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fourth_quarter_drives = drive_data[(drive_data['qtr']==4)&(abs(pbp_clean['score_differential']) <= 8 )]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:100: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fullteamdata['first_down_passrun_o

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:276: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['off_bad_drives_perdrive'] = (drive_data[(drive_data['fixed_drive_result']!='Touchdown') & (drive_data['fixed_drive_result']!='Field Goal')].groupby('posteam')['play_id'].count().reset_index()['play_id']/off_drive_count_data['drive_count'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:277: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['def_bad_

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:319: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['firstdowns_pergame_off_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('posteam')['first_down'].sum()/reggame_count_data).reset_index()[0]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:320: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['firstdowns_pergame_def_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('defteam')['fi

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:408: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['epa/db_diff'] = ((qb1_seasonstats['passing_epa']/qb1_seasonstats['attempts']) - pbp_rp[pbp_rp['pass']==1].groupby('posteam')['epa'].mean()).reset_index()[0]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:411: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['comppct_diff'] = ((qb1_seasonstats['completions']/qb1_seasonstats['attempts']) - fullteamda

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:244: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['penalties'] = penalties_data['play_id']
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:245: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  penaltyyards_data = pbp_temp.groupby('penalty_team').sum()['penalty_yards'].reset_index()
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:246: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling

...

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:408: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['epa/db_diff'] = ((qb1_seasonstats['passing_epa']/qb1_seasonstats['attempts']) - pbp_rp[pbp_rp['pass']==1].groupby('posteam')['epa'].mean()).reset_index()[0]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:411: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['comppct_diff'] = ((qb1_seasonstats['completions']/qb1_seasonstats['attempts']) - fullteamda

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:244: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['penalties'] = penalties_data['play_id']
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:245: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  penaltyyards_data = pbp_temp.groupby('penalty_team').sum()['penalty_yards'].reset_index()
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:246: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:314: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['yds_pergame_def_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('defteam')['yards_gained'].sum()/reggame_count_data).reset_index()[0]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:317: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['firstdowns_pergame_off'] = (pbp_clean.groupby('posteam')['first_down'].sum()/game_count_data).reset_in

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:408: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['epa/db_diff'] = ((qb1_seasonstats['passing_epa']/qb1_seasonstats['attempts']) - pbp_rp[pbp_rp['pass']==1].groupby('posteam')['epa'].mean()).reset_index()[0]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/688748034.py:411: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fullteamdata['comppct_diff'] = ((qb1_seasonstats['completions']/qb1_seasonstats['attempts']) - fullteamda

,team,off_epa,off_wpa,def_epa,def_wpa,offrp_epa,offrp_wpa,defrp_epa,defrp_wpa,off4min_epa,...,postthxgiving_off_comp_percentage,postthxgiving_def_comp_percentage,epa/db_diff,comppct_diff,td-int_diff,ypa_diff,avg_exp_2021,avg_exp_2022,exp_diff,wins
0,ARI,0.074580,0.001948,0.071577,0.002838,0.067081,0.000617,0.055624,0.002501,0.114147,...,0.613900,0.607656,0.012432,0.077501,-2.0,0.599539,3.202020,3.216495,-0.014475,8
1,ATL,0.018536,0.001329,0.073925,0.002940,0.034196,0.001195,0.061149,0.003386,0.032593,...,0.563107,0.563025,-0.001969,0.058136,-3.0,0.868091,3.654206,3.591398,0.062808,8
2,BAL,0.185912,0.003941,0.004111,0.000892,0.219023,0.004447,-0.038662,0.000224,0.260048,...,0.563380,0.547414,0.031697,0.080976,-3.0,0.625315,3.258427,3.422222,-0.163795,13
3,BUF,0.169401,0.003862,0.021687,0.000624,0.189500,0.004286,0.008599,-0.000166,0.157621,...,0.585366,0.588785,-0.017049,0.060574,-9.0,0.400715,3.479167,3.504762,-0.025595,15
4,CAR,-0.029567,0.001379,0.150534,0.004006,-0.053179,0.001242,0.159538,0.003776,0.056937,...,0.519149,0.575472,0.003086,0.066381,-8.0,0.714819,3.414414,2.752475,0.661939,5
5,CHI,-0.049304,-0.001083,0.021739,0.001608,-0.081907,-0.002215,0.005892,0.001653,0.086975,...,0.510638,0.622449,-0.020035,0.112961,1.0,1.252885,3.288889,3.255319,0.033570,5
6,CIN,0.091495,0.004163,0.058187,0.003521,0.090178,0.004275,0.037274,0.002322,0.204906,...,0.686347,0.579399,0.014345,0.069284,-4.0,0.8417,2.570000,3.053191,-0.483191,9
7,CLE,-0.150758,-0.002820,0.013101,0.001071,-0.183319,-0.003443,0.017447,0.000882,-0.167742,...,0.505415,0.537383,-0.076579,0.121272,2.0,0.462867,3.562500,3.284314,0.278186,3
8,DAL,-0.046303,0.000908,0.087355,0.002646,-0.109425,0.000033,0.083002,0.002803,-0.064929,...,0.570732,0.592275,0.076243,0.067523,-8.0,1.19556,3.373737,3.433333,-0.059596,7
9,DEN,0.044571,0.000933,-0.055063,-0.000052,0.014592,0.000351,-0.090583,-0.000540,-0.015817,...,0.600917,0.554264,-0.001544,0.079187,-4.0,0.469141,2.915663,3.011236,-0.095573,10


In [10]:
# transform functions
def transform(df_temp, df_this_season = None):
  df_clean = None
  if df_this_season is None:
      df_clean = df_temp[(df_temp['play_type'] != 'qb_kneel') & (df_temp['play_type'] != 'no_play') & (df_temp['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
  else:
    df_clean = pd.concat([df_temp, df_this_season], ignore_index=True)
#     df_clean = df_clean.dropna(axis=1, how='any')
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
  pbp_rp_transform = df_clean[(df_clean['pass']==1)|(df_clean['rush']==1)]
  second_down_transform = df_clean[df_clean['down']==2]
  third_down_transform = df_clean[df_clean['down']==3]
  df[['offrp_epa','offrp_wpa']] = (pbp_rp_transform.groupby('posteam').mean()[['epa','wpa',]])
  df[['defrp_epa','defrp_wpa']] = (pbp_rp_transform.groupby('defteam').mean()[['epa','wpa',]])
  df['ydstogo_2nd_off'] = (second_down_transform.groupby('posteam').mean()[['ydstogo']])
  df['ydstogo2nd_def'] = (second_down_transform.groupby('defteam').mean()[['ydstogo']])
  df['ydstogo_3rd_off'] = (third_down_transform.groupby('posteam').mean()[['ydstogo']])
  df['ydstogo_3rd_def'] = (third_down_transform.groupby('defteam').mean()[['ydstogo']])
  df['third_down_off'] = (third_down_transform.groupby('posteam').mean()[['third_down_converted']])
  df['third_down_def'] = (third_down_transform.groupby('defteam').mean()[['third_down_converted']])
  df['ydsperplay'] = (pbp_rp_transform.groupby('posteam').mean()['yards_gained'])
  df['ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('posteam').mean()['yards_gained'])
  df['ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('posteam').mean()['yards_gained'])
  df['opp_ydsperplay'] = (pbp_rp_transform.groupby('defteam').mean()['yards_gained'])
  df['opp_ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('defteam').mean()['yards_gained'])
  df['opp_ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('defteam').mean()['yards_gained'])
  return df

# transform(pbp_2022[(pbp_2022['week']<5)&(pbp_2022['week']>=3)], 'TB','DAL')

# 'TB' not in list(win_totals_2022[win_totals_2022['week']==11]['away_team']) and \
#       'TB' not in list(win_totals_2022[win_totals_2022['week']==11]['home_team'])

In [11]:
###thisseason method
def thisseason(pbp_temp, schedule_temp):
  df_clean = pbp_temp[(pbp_temp['play_type'] != 'qb_kneel') & (pbp_temp['play_type'] != 'no_play') & (pbp_temp['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
  pbp_rp_this = df_clean[(df_clean['pass']==1)|(df_clean['rush']==1)]
  second_down_this = df_clean[df_clean['down']==2]
  third_down_this = df_clean[df_clean['down']==3]
  fourq_this = df_clean[df_clean['qtr']==4]
  groupby_away = schedule_temp.groupby('away_team')
  groupby_home = schedule_temp.groupby('home_team')
  schedule_temp['away_win'] = schedule_temp.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
  schedule_temp['home_win'] = schedule_temp.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
  game_count_this = groupby_home['week'].nunique()+groupby_away['week'].nunique()
  #yards per play(including fourth quarter)
  df = df_clean.groupby('posteam')[['yards_gained']].mean().rename(columns={'yards_gained':'ydsperplay'})
  df['ydsperattempt'] = df_clean[df_clean['pass'] == 1].groupby('posteam')['yards_gained'].mean()
  df['ydsperrush'] = df_clean[df_clean['rush'] == 1].groupby('posteam')['yards_gained'].mean()
  df['opp_ydsperplay'] = df_clean.groupby('defteam')['yards_gained'].mean()
  df['opp_ydsperattempt'] = df_clean[df_clean['pass'] == 1].groupby('defteam')['yards_gained'].mean()
  df['opp_ydsperrush'] = df_clean[df_clean['rush'] == 1].groupby('defteam')['yards_gained'].mean()
  #4q
  df['ydsperplay4q'] = fourq_this.groupby('posteam')['yards_gained'].mean()
  df['opp_ydsperplay4q'] = fourq_this.groupby('defteam')['yards_gained'].mean()

  #epa and wpa(including 4q)
  df[['epa_off','wpa_off']] = df_clean.groupby('posteam')[['epa','wpa']].mean()
  df[['epa_def','wpa_def']] = df_clean.groupby('defteam')[['epa', 'wpa']].mean()
  df[['epa_off_4q','wpa_off_4q']] = fourq_this.groupby("posteam")[['epa', 'wpa']].mean()
  df[['epa_def_4q','wpa_def_4q']] = fourq_this.groupby("defteam")[['epa', 'wpa']].mean()
  #down stats
  df['2ndydstogo_off'] = second_down_this.groupby('posteam')['ydstogo'].mean()
  df['2ndydstogo_def'] = second_down_this.groupby('defteam')['ydstogo'].mean()
  df['3rdydstogo_off'] = third_down_this.groupby('posteam')['ydstogo'].mean()
  df['3rdydstogo_def'] = third_down_this.groupby('defteam')['ydstogo'].mean()
  df['3rddownconverted_off'] = third_down_this.groupby('posteam')['third_down_converted'].mean()
  df['3rddownconverted_def'] = third_down_this.groupby('defteam')['third_down_converted'].mean()
  #points per game
  df['points_pergame_off'] = ((groupby_home['home_score'].sum()+groupby_away['away_score'].sum())/game_count_this)
  df['points_pergame_def'] = ((groupby_home['away_score'].sum()+groupby_away['home_score'].sum())/game_count_this)
  #yards per game
  df['yards_pergame_off'] = ((df_clean.groupby('posteam')['yards_gained'].sum())/game_count_this)
  df['yards_pergame_def'] = ((df_clean.groupby('defteam')['yards_gained'].sum())/game_count_this)
  #win percentage
  df['win_percentage']=((groupby_home['home_win'].sum()+groupby_away['away_win'].sum())/game_count_this)
  #average point differential
  df['off_pointdifferential'] = df_clean.groupby('posteam')['score_differential'].mean()
  df['def_pointdifferential'] = (df_clean.groupby('defteam')['score_differential'].mean() * (-1))
  #completion percentage
  df['off_comp_percentage'] = df_clean[df_clean['pass'] == 1].groupby('posteam')['complete_pass'].mean()
  df['def_comp_percentage'] = df_clean[df_clean['pass'] == 1].groupby('defteam')['complete_pass'].mean()

  return df.add_suffix("_thisyear")


In [12]:
### organization, create an list of dataframes for each game


game_list_prev = []
results = []
this_season = fullteamdata_arr.pop(current_year - 2020)
year = 2020
count = 0
for fullteamdata in fullteamdata_arr:
  pbp_thisyear=pbp_data[pbp_data['season']==year]
  pbp_lastyear=pbp_data[pbp_data['season']==year-1]
  pbp_lastyear=pbp_lastyear[pbp_lastyear['season_type'] == 'REG']
  win_totals_temp = win_totals[win_totals['season']==year]
  win_totals_lastyear = win_totals[win_totals['season']==year]
  win_totals_lastyear = win_totals[win_totals['game_type']=='REG']
  week1_df = pd.DataFrame(index=fullteamdata.columns)
  fullteamdata.set_index('team',inplace=True)
#   thanksgiving = fullteamdata.loc[:,'postthxgiving_ydsperplay':'postthxgiving_def_comp_percentage']
  thanksgiving = fullteamdata.iloc[:,176:207]

  thanksgiving.columns = thanksgiving.columns.str.replace('postthxgiving_','')
  thanksgiving = thanksgiving.add_suffix('_thisyear')
  for i in range(1, 23, 1):
    df_temp_thisyear = None
    win_totals_temp_thisyear = None
    if i > 1 and i <= 4:
        df_temp_thisyear = pbp_thisyear[pbp_thisyear['week']<i]
        win_totals_temp_thisyear = win_totals_temp[win_totals_temp['week']<i]
    if i <= 4:
      df_temp=pbp_lastyear[pbp_lastyear['week']>(i*2 + 8)]
      win_totals_temp_lastyear = win_totals_lastyear[win_totals_lastyear['week']>(i*2+8)]
      df_past4weeks=transform(df_temp, df_temp_thisyear)
      
      if i == 1:
        df_thisseason = thisseason(df_temp, win_totals_temp_lastyear)
      else:
        # concat dataframes
        thisseason_first_param = pd.concat([df_temp, df_temp_thisyear], ignore_index=True)
        thisseason_second_param = pd.concat([win_totals_temp_lastyear, win_totals_temp_thisyear], ignore_index=True)
        df_thisseason = thisseason(thisseason_first_param, thisseason_second_param)
#         df_thisseason = thanksgiving
        
        
    else:
      df_thisseason = thisseason(pbp_thisyear[pbp_thisyear['week']<i], win_totals_temp[win_totals_temp['week']<i])
      df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-5)]
      df_past4weeks=transform(df_temp)
    df_past4weeks=df_past4weeks.add_suffix('_4weeks')
    if i <= 3:
      df_temp=pbp_lastyear[pbp_lastyear['week']>11+i]
      df_past3weeks=transform(df_temp, df_temp_thisyear)
    else:
      df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-3)]
      df_past3weeks=transform(df_temp)
    df_past3weeks=df_past3weeks.add_suffix('_3weeks')
    if i <= 2:
      df_temp=pbp_lastyear[pbp_lastyear['week']>13+i]
      df_past2weeks=transform(df_temp, df_temp_thisyear)
    else:
      df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-2)]
      df_past2weeks=transform(df_temp)
    df_past2weeks=df_past2weeks.add_suffix('_2weeks')

    df_thisseason.fillna(0,inplace=True)
    fullteamdata_joined = fullteamdata.join(df_thisseason.join(df_past4weeks.join(df_past3weeks.join(df_past2weeks))))
#     thisyear_temp = fullteamdata_joined.loc[:,'ydsperplay_thisyear':'def_comp_percentage_thisyear']
    thisyear_temp = fullteamdata_joined.iloc[:,215:246]

    thisyear_temp.columns = thisyear_temp.columns.str.replace('_thisyear','')
#     postthxgiving_temp = fullteamdata_joined.loc[:,'postthxgiving_ydsperplay':'postthxgiving_def_comp_percentage']
    postthxgiving_temp = fullteamdata_joined.iloc[:,176:207]

    postthxgiving_temp.columns = postthxgiving_temp.columns.str.replace('postthxgiving_','')
    subtract = thisyear_temp.subtract(postthxgiving_temp).add_suffix('_difference')
    fullteamdata_joined = fullteamdata_joined.join(subtract)
    #make the time list
    # time_week = []
    # for a in range(0,378):
    #   time_week.append(i)
    # time_week

    for j in win_totals_temp[win_totals_temp['week']==i].index:
      away = fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']]
      home = fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']]
      df_away = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']].add_suffix('_away')).rename(columns={win_totals_temp.loc[j, 'away_team']:'data'})
      df_home = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']].add_suffix('_home')).rename(columns={win_totals_temp.loc[j, 'home_team']:'data'})
      df_difference = pd.DataFrame(away.subtract(home).add_suffix('_difference')).rename(columns={0:'data'})
      df = pd.concat([df_away,pd.concat([df_home,df_difference])])
      df = pd.concat([df, pd.DataFrame([i],index=['week'],columns=['data'])])
      df.fillna(0,inplace=True)
      count += 1

#       if len(df.dropna(axis=0).index) < 1021:
#         #column of error
#         count_arr.append(count)



      df_list = list(df['data'])





      results.append([win_totals_temp.loc[j,'away_score'],win_totals_temp.loc[j,'home_score']])
      game_list_prev.append(df_list)
  year+=1
  fullteamdata.reset_index(inplace=True)

fullteamdata_arr.append(this_season)
# team = 'ARI'
# fullteamdata_2021_reset[fullteamdata_2021_reset['team']==team].reset_index()
# fullteamdata_2021.loc[team]
# week1_df =
len(game_list_prev)
# fullteamdata_2021
# game_list_2022
# game_list_2022

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['offrp_epa','offrp_wpa']] = (pbp_rp_transform.groupby('posteam').mean()[['epa','wpa',]])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['defrp_epa','defrp_wpa']] = (pbp_rp_transform.groupby('defteam').mean()[['epa','wpa',]])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('posteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:25: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['opp_ydsperplay'] = (pbp_rp_transform.groupby('defteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is dep

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

...

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/1082027448.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_temp['away_win'] = schedule_temp.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/1082027448.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_temp['home_win'] = schedule_temp.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('posteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('posteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:25: FutureWarning: The default value of numeric_only in 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/1082027448.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_temp['away_win'] = schedule_temp.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/1082027448.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_temp['home_win'] = schedule_temp.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['offrp_epa','offrp_wpa']] = (pbp_rp_transform.groupby('posteam').mean()[['epa','wpa',]])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['defrp_epa','defrp_wpa']] = (pbp_rp_transform.groupby('defteam').mean()[['epa','wpa',]])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated

1408

In [13]:
this_year_schedule = nfl.import_schedules([current_year])
sundays = pd.to_datetime(this_year_schedule[this_year_schedule['weekday'] == 'Sunday']['gameday']).unique()[:-1]
tuesdays = sundays + pd.Timedelta(days=2, hours=7)
dates_df = pd.DataFrame({'Tuesdays': tuesdays, 'Week': range(1,len(tuesdays)+1)})
today = pd.to_datetime(datetime.today())
dates_df = dates_df[dates_df['Tuesdays'] > today]
current_week = None
if len(dates_df) == 0:
    current_week = int(this_year_schedule.tail(1)['Week'])
else:
    current_week = int(dates_df.head(1)['Week'])
    

In [14]:
### create a list of dataframes for each game(2024)
game_list_2024 = []
training_data = []
results_2024 = []
# week1_df = pd.DataFrame(index=this_year.columns)
this_season.set_index('team',inplace=True)
pbp_thisyear=pbp_data[pbp_data['season']==current_year]
pbp_lastyear=pbp_data[pbp_data['season']==current_year-1]
pbp_lastyear=pbp_lastyear[pbp_lastyear['season_type'] == 'REG']

win_totals_temp = win_totals[win_totals['season']==current_year]
win_totals_lastyear = win_totals[win_totals['season']==year]
win_totals_lastyear = win_totals[win_totals['game_type']=='REG']
# win_totals_2021[win_totals_2021['week']==1].index
# count_bye = 0
# count_byetwoweeks = 0
# thanksgiving = fullteamdata.loc[:,'postthxgiving_ydsperplay':'postthxgiving_def_comp_percentage']
thanksgiving = this_season.iloc[:,176:207]

thanksgiving.columns = thanksgiving.columns.str.replace('postthxgiving_','')
thanksgiving = thanksgiving.add_suffix('_thisyear')

###change this value!!!
# current_week = 2
####
temp_thisseason = None
temp_past5 = None
temp_df = None

for i in range(1, current_week+1, 1):
  df_temp_thisyear = None
  win_totals_temp_thisyear = None
  if i > 1 and i <= 4:
    df_temp_thisyear = pbp_thisyear[pbp_thisyear['week']<i]  
    win_totals_temp_thisyear = win_totals_temp[win_totals_temp['week']<i]
  if i <= 4:
#     df_thisseason = thanksgiving
    df_temp=pbp_lastyear[pbp_lastyear['week']>(i*2+8)]
    win_totals_temp_lastyear = win_totals_lastyear[win_totals_lastyear['week']>(i*2+8)]

    df_past5weeks=transform(df_temp,df_temp_thisyear)
    if i == 1:
        df_thisseason = thisseason(df_temp, win_totals_temp_lastyear)
    else:
        # concat dataframes
        thisseason_first_param = pd.concat([df_temp, df_temp_thisyear], ignore_index=True)
        thisseason_second_param = pd.concat([win_totals_temp_lastyear, win_totals_temp_thisyear], ignore_index=True)
        df_thisseason = thisseason(thisseason_first_param, thisseason_second_param)

  else:
    df_thisseason = thisseason(pbp_thisyear[pbp_thisyear['week']<i], win_totals_temp[win_totals_temp['week']<i])
    df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-5)]
    df_past5weeks=transform(df_temp)
  df_past5weeks=df_past5weeks.add_suffix('_5weeks')
  if i <= 3:
    df_temp=pbp_lastyear[pbp_lastyear['week']>11+i]
    df_past3weeks=transform(df_temp,df_temp_thisyear)
  else:
    df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-3)]
    df_past3weeks=transform(df_temp)
  df_past3weeks=df_past3weeks.add_suffix('_3weeks')
  if i <= 2:
    df_temp=pbp_lastyear[pbp_lastyear['week']>13+i]
    df_past2weeks=transform(df_temp,df_temp_thisyear)
  else:
    df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-2)]
    df_past2weeks=transform(df_temp)
  df_past2weeks=df_past2weeks.add_suffix('_2weeks')
  # if i == 5:
  #   print(df_thisseason.iloc[:,30:40])
  df_thisseason.fillna(0)
  fullteamdata_joined = this_season.join(df_thisseason.join(df_past5weeks.join(df_past3weeks.join(df_past2weeks))))
#   thisyear_temp = fullteamdata_joined.loc[:,'ydsperplay_thisyear':'def_comp_percentage_thisyear']
  thisyear_temp = fullteamdata_joined.iloc[:,215:246]
  
  thisyear_temp.columns = thisyear_temp.columns.str.replace('_thisyear','')
#   postthxgiving_temp = fullteamdata_joined.loc[:,'postthxgiving_ydsperplay':'postthxgiving_def_comp_percentage']
  postthxgiving_temp = fullteamdata_joined.iloc[:,176:207]

  postthxgiving_temp.columns = postthxgiving_temp.columns.str.replace('postthxgiving_','')
  subtract = thisyear_temp.subtract(postthxgiving_temp).add_suffix('_difference')
  fullteamdata_joined = fullteamdata_joined.join(subtract)
  #make the time list

  for j in win_totals_temp[win_totals_temp['week']==i].index:
    away = fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']]
    home = fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']]
    df_away = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']].add_suffix('_away')).rename(columns={win_totals_temp.loc[j, 'away_team']:'data'})
    df_home = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']].add_suffix('_home')).rename(columns={win_totals_temp.loc[j, 'home_team']:'data'})
    df_difference = pd.DataFrame(away.subtract(home).add_suffix('_difference')).rename(columns={0:'data'})
    df = pd.concat([df_away,pd.concat([df_home,df_difference])])
    df = pd.concat([df, pd.DataFrame([i],index=['week'],columns=['data'])])
    df_list = list(df['data'])
    if i == 1:
        temp_df = df


    if i == current_week:
      game_list_2024.append(df_list)
    else:
      training_data.append(df_list)
      results_2024.append([win_totals_temp.loc[j,'away_score'],win_totals_temp.loc[j,'home_score']])

this_season.reset_index(inplace=True)


# team = 'ARI'
# fullteamdata_2021_reset[fullteamdata_2021_reset['team']==team].reset_index()
# fullteamdata_2021.loc[team]
# week1_df =

len(game_list_2024)
# fullteamdata_2021
# game_list_2022
# game_list_2022

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:26: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['opp_ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('defteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['opp_ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('defteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_o

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. 

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/1082027448.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_temp['away_win'] = schedule_temp.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/1082027448.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule_temp['home_win'] = schedule_temp.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['third_down_def'] = (third_down_transform.groupby('defteam').mean()[['third_down_converted']])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperplay'] = (pbp_rp_transform.groupby('posteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a f

/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:22: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperplay'] = (pbp_rp_transform.groupby('posteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('posteam').mean()['yards_gained'])
/var/folders/rj/tzk4w23s0md7pcnpq4w2fy6c0000gn/T/ipykernel_41741/3731504997.py:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is depr

15

In [15]:
df_thisseason.iloc[:,8:]

,epa_off_thisyear,wpa_off_thisyear,epa_def_thisyear,wpa_def_thisyear,epa_off_4q_thisyear,wpa_off_4q_thisyear,epa_def_4q_thisyear,wpa_def_4q_thisyear,2ndydstogo_off_thisyear,2ndydstogo_def_thisyear,...,3rddownconverted_def_thisyear,points_pergame_off_thisyear,points_pergame_def_thisyear,yards_pergame_off_thisyear,yards_pergame_def_thisyear,win_percentage_thisyear,off_pointdifferential_thisyear,def_pointdifferential_thisyear,off_comp_percentage_thisyear,def_comp_percentage_thisyear
posteam,,,,,,,,,,,,,,,,,,,,,
ARI,-0.005325,0.001129,-0.019454,0.003181,-0.127115,-0.001405,0.159845,0.011755,8.509615,7.342593,...,0.343284,20.60,19.20,288.80,348.6,0.40,3.100817,5.198979,0.567010,0.584475
ATL,-0.019673,0.001314,0.009130,0.001565,0.098006,0.003692,0.058835,0.003013,7.428571,7.623188,...,0.340909,19.00,21.50,364.00,245.0,0.50,-1.358065,1.463035,0.576389,0.483871
BAL,0.083316,0.001335,0.181250,0.004988,-0.082749,-0.004941,0.192527,0.008447,7.720000,7.096491,...,0.458333,28.20,35.40,312.60,411.0,0.20,-4.274692,-0.178147,0.587097,0.623810
BUF,0.168416,0.005599,0.069817,0.002705,0.226764,0.009485,0.027384,0.001710,6.944445,7.977012,...,0.406780,30.60,22.60,398.20,299.6,0.80,1.313158,5.251429,0.586957,0.540881
CAR,-0.034339,-0.000939,0.049348,-0.001196,0.214792,0.002184,-0.082254,-0.001408,7.106796,8.565217,...,0.381818,20.40,23.80,316.20,312.8,0.40,-8.259259,-4.317365,0.563452,0.595238
CHI,-0.014680,0.001232,0.009428,0.000191,-0.031949,-0.000187,0.137954,0.001442,9.312500,6.202703,...,0.292683,25.25,29.25,328.75,380.5,0.50,-2.806557,0.907801,0.538961,0.672000
CIN,-0.170367,-0.001938,0.115115,0.000274,-0.085516,-0.004781,-0.137108,-0.012380,7.852273,7.637931,...,0.475410,17.00,31.20,228.80,392.2,0.40,-10.937685,-8.566416,0.558011,0.621762
CLE,-0.163402,-0.004091,-0.013293,-0.000777,-0.351281,-0.013587,-0.217568,-0.010428,8.176991,7.717391,...,0.432836,14.60,24.60,288.40,249.4,0.20,-6.260000,-3.329412,0.550239,0.581250
DAL,0.160551,0.010314,0.193466,0.010255,0.144466,0.014857,0.340105,0.020054,7.243243,7.551402,...,0.549296,30.20,30.80,407.20,414.2,0.40,-2.118812,1.800000,0.666667,0.629808


In [16]:
df_past5weeks

,off_epa_5weeks,off_wpa_5weeks,def_epa_5weeks,def_wpa_5weeks,offrp_epa_5weeks,offrp_wpa_5weeks,defrp_epa_5weeks,defrp_wpa_5weeks,ydstogo_2nd_off_5weeks,ydstogo2nd_def_5weeks,ydstogo_3rd_off_5weeks,ydstogo_3rd_def_5weeks,third_down_off_5weeks,third_down_def_5weeks,ydsperplay_5weeks,ydsperattempt_5weeks,ydsperrush_5weeks,opp_ydsperplay_5weeks,opp_ydsperattempt_5weeks,opp_ydsperrush_5weeks
posteam,,,,,,,,,,,,,,,,,,,,
ARI,-0.005325,0.001129,-0.019454,0.003181,-0.006700,0.001607,-0.019532,0.000200,8.509615,7.342593,7.485294,7.238806,0.397059,0.343284,4.813334,5.118556,4.254717,5.281818,6.031963,3.801802
ATL,-0.019673,0.001314,0.009130,0.001565,-0.015406,0.001692,-0.039926,0.000529,7.428571,7.623188,6.018182,6.409091,0.418182,0.340909,5.621622,6.534722,4.478261,4.757282,5.129032,4.195122
BAL,0.083316,0.001335,0.181250,0.004988,0.050563,0.001067,0.188904,0.004410,7.720000,7.096491,7.647059,6.277778,0.411765,0.458333,6.302419,6.909678,5.290323,5.939306,6.633333,4.867647
BUF,0.168416,0.005599,0.069817,0.002705,0.190281,0.004472,0.021045,0.001503,6.944445,7.977012,6.125000,7.677966,0.464286,0.406780,6.300633,7.570652,4.530303,5.388489,5.666667,5.016807
CAR,-0.034339,-0.000939,0.049348,-0.001196,-0.028208,-0.000830,0.056930,0.001045,7.106796,8.565217,6.567164,7.254546,0.388060,0.381818,4.956113,5.045685,4.811475,5.771218,6.517857,4.553398
CHI,-0.014680,0.001232,0.009428,0.000191,-0.025371,0.000347,0.009026,0.000821,9.312500,6.202703,8.535714,6.634146,0.446429,0.292683,5.345529,6.551948,3.326087,6.675438,7.280000,5.941748
CIN,-0.170367,-0.001938,0.115115,0.000274,-0.242400,-0.002003,0.120902,0.000798,7.852273,7.637931,7.704918,6.131147,0.360656,0.475410,4.300752,4.928177,2.964706,5.853731,6.813471,4.549296
CLE,-0.163402,-0.004091,-0.013293,-0.000777,-0.167314,-0.005188,-0.031296,0.000684,8.176991,7.717391,6.973684,6.776119,0.342105,0.432836,4.382979,4.698565,3.833333,4.567766,5.575000,3.141593
DAL,0.160551,0.010314,0.193466,0.010255,0.165638,0.006574,0.205952,0.008822,7.243243,7.551402,6.271186,7.183099,0.440678,0.549296,6.188450,6.619718,5.396552,6.275758,7.500000,4.188525


In [17]:
win_totals_temp[win_totals_temp['week']==current_week-1]

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,away_win,home_win
7055,2025_05_SF_LA,2025,REG,5,2025-10-02,Thursday,20:15,SF,26.0,LA,...,00-0026498,Mac Jones,Matthew Stafford,Kyle Shanahan,Sean McVay,Bill Vinovich,LAX01,SoFi Stadium,1,0
7056,2025_05_MIN_CLE,2025,REG,5,2025-10-05,Sunday,09:30,MIN,21.0,CLE,...,00-0040704,Carson Wentz,Dillon Gabriel,Kevin O'Connell,Kevin Stefanski,Scott Novak,CLE00,FirstEnergy Stadium,1,0
7057,2025_05_HOU_BAL,2025,REG,5,2025-10-05,Sunday,13:00,HOU,44.0,BAL,...,00-0033662,C.J. Stroud,Cooper Rush,DeMeco Ryans,John Harbaugh,Land Clark,BAL00,M&T Bank Stadium,1,0
7058,2025_05_MIA_CAR,2025,REG,5,2025-10-05,Sunday,13:00,MIA,24.0,CAR,...,00-0039150,Tua Tagovailoa,Bryce Young,Mike McDaniel,Dave Canales,Clay Martin,CAR00,Bank of America Stadium,0,1
7059,2025_05_LV_IND,2025,REG,5,2025-10-05,Sunday,13:00,LV,6.0,IND,...,00-0035710,Geno Smith,Daniel Jones,Pete Carroll,Shane Steichen,Alex Kemp,IND00,Lucas Oil Stadium,0,1
7060,2025_05_NYG_NO,2025,REG,5,2025-10-05,Sunday,13:00,NYG,14.0,NO,...,00-0039376,Jaxson Dart,Spencer Rattler,Brian Daboll,Kellen Moore,Alan Eck,NOR00,Mercedes-Benz Superdome,0,1
7061,2025_05_DAL_NYJ,2025,REG,5,2025-10-05,Sunday,13:00,DAL,37.0,NYJ,...,00-0036945,Dak Prescott,Justin Fields,Brian Schottenheimer,Aaron Glenn,Alex Moore,NYC01,MetLife Stadium,1,0
7062,2025_05_DEN_PHI,2025,REG,5,2025-10-05,Sunday,13:00,DEN,21.0,PHI,...,00-0036389,Bo Nix,Jalen Hurts,Sean Payton,Nick Sirianni,Adrian Hill,PHI00,Lincoln Financial Field,1,0
7063,2025_05_TEN_ARI,2025,REG,5,2025-10-05,Sunday,16:05,TEN,22.0,ARI,...,00-0035228,Cam Ward,Kyler Murray,Brian Callahan,Jonathan Gannon,Shawn Smith,PHO00,State Farm Stadium,1,0
7064,2025_05_TB_SEA,2025,REG,5,2025-10-05,Sunday,16:05,TB,38.0,SEA,...,00-0034869,Baker Mayfield,Sam Darnold,Todd Bowles,Mike Macdonald,Craig Wrolstad,SEA00,Lumen Field,1,0


In [22]:
## the model


sum_arr = np.array([])

#concatenate past year's data and this year's data
training_x = game_list_prev + training_data
training_y = results + results_2024

# training_x = training_x[269:]
# training_y = training_y[269:]
### create and fit the model
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score


for i in range(0,10):

  ### play around with hidden layer sizes and check validation results
  # if current_week < 5:
  #   prediction_model = MLPRegressor(hidden_layer_sizes=(200,110,60), max_iter=17)
  # else:
  #   prediction_model = MLPRegressor(hidden_layer_sizes=(195,110,60), max_iter=17)

#   prediction_model = MLPRegressor(hidden_layer_sizes=(195,110,60), activation='relu',max_iter=17)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(195,110,60), activation='relu',max_iter=50)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(175,90,50), activation='relu',max_iter=50)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(175,90,50), activation='relu',max_iter=78)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(245,125,65), activation='relu',max_iter=50)

### High max iter:
#   prediction_model = MLPRegressor(hidden_layer_sizes=(245,125,65), activation='relu',max_iter=110)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(175,90,50), activation='relu',max_iter=110)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(150,75,39), activation='relu',max_iter=110)
#   prediction_model = MLPRegressor(hidden_layer_sizes=(100,50,25), activation='relu',max_iter=110)
  prediction_model = MLPRegressor(hidden_layer_sizes=(150,75,39), activation='relu',max_iter=50)



  prediction_model.fit(training_x,training_y)#185,110,60,  16
  ###validation
  pred = prediction_model.predict(game_list_prev)
  r2score = r2_score(results, pred)
  # prediction_model.predict([first_game])
  print(r2score)
  # pred
  if i > 0:
    sum_arr = np.array(prediction_model.predict(game_list_2024)) + sum_arr
  else:
    sum_arr = np.array(prediction_model.predict(game_list_2024))
  time.sleep(0.1)



arr = sum_arr/10

/Users/michaelzheng119/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.2533966520698685


/Users/michaelzheng119/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.2165043700759856


/Users/michaelzheng119/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


...

0.2519995256340868


/Users/michaelzheng119/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.23695571583834418


/Users/michaelzheng119/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.18472355374109428
0.23658869619879053


In [23]:
#average point differential for the home team
def superbowl(arr):
    # Calculate home field advantage
    sum_test = 0
    for homefield_arr in training_y:
      sum_test += homefield_arr[1] - homefield_arr[0]
    home_advantage = sum_test/len(training_y)
    home_advantage
    
    # only use for super bowl (neutral site):

    arr[0][0] = arr[0][0] + home_advantage/2
    arr[0][1] = arr[0][1] - home_advantage/2
    df = pd.DataFrame(arr, columns=['away_score', 'home_score'])
    df.insert(0, 'game', win_totals_temp[win_totals_temp['week']==current_week][['game_id']].set_index(df.index))
    return df

In [24]:
### predictions in table format
current_week_schedule = this_year_schedule[this_year_schedule['week'] == current_week]

# Check if game is the super bowl
if len(current_week_schedule) == 1 and current_week_schedule['game_type'] == 'SB':
    df = superbowl(arr)
else:
    df = pd.DataFrame(arr, columns=['away_score', 'home_score'])
    df.insert(0, 'game', win_totals_temp[win_totals_temp['week']==current_week][['game_id']].set_index(df.index))
df

,game,away_score,home_score
0,2025_06_PHI_NYG,25.382658,18.412232
1,2025_06_DEN_NYJ,23.326570,19.498180
2,2025_06_LA_BAL,26.975269,23.047417
3,2025_06_DAL_CAR,22.408773,22.524004
4,2025_06_ARI_IND,17.315200,27.806286
5,2025_06_SEA_JAX,24.167584,21.755082
6,2025_06_LAC_MIA,22.490092,20.457711
7,2025_06_NE_NO,26.079538,22.440323
8,2025_06_CLE_PIT,20.255878,26.698100
9,2025_06_TEN_LV,16.008345,23.773993


In [21]:

###test
# depth_chart[(depth_chart['week'] == 1)&(depth_chart['club_code']=='TB')]
# depth_chart.iloc[:,20:30]
#multiple years?
# pbp_list = nfl.import_pbp_data([2015,2016,2017,2018,2019,2020,2021,2022])
# depth_chart[(depth_chart['position']=='QB')&(depth_chart['club_code']=='NE')&(depth_chart['week']==22)]
# pbp_list
#rookie case
# seasonal_data[seasonal_data['player_id']==00000]['completions']
# pbp_rp_2021[pbp_rp_2021['pass']==1].groupby('posteam')['epa'].mean()

# postthxgiving_temp['epa/db_diff']
